In [5]:
from py_ecc.bn128 import G1, G2, multiply, add, curve_order, eq, Z1, pairing, neg
import numpy as np
import galois   

In [6]:
GF = galois.GF(curve_order)
# takes about 2 minutes

In [68]:
x = GF(5)#random.randint(1,10)
y = GF(10)#random.randint(1,15)

v1 = x * x
v2 = v1 * x
v3 = y * y
v4 = v2 * y
out = v2 + GF(2) * v4 - GF(5) * x * v3 - GF(3) * y + GF(2)

L = GF(np.array([
    [0,0,1,0,0,0,0,0],
    [0,0,0,0,1,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,curve_order - 5,0,0,0,0,0] # 113 - 5 = 108
    ]))

R = GF(np.array([
    [0,0,1,0,0,0,0,0],
    [0,0,1,0,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,0,1,0]
    ]))

O = GF(np.array([
    [0,0,0,0,1,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,1],
    [curve_order-2,1,0,3,0,curve_order-1,0,curve_order-2] # 113 - 2 = 111
    ]))

witness = GF(np.array([GF(1),out,x,y,v1,v2,v3,v4]))
print(f"witness= {type(witness)}")
print(f"generated L, R, O R1CS matrices and witness vector")
print(f"verifying that L.w * R.w == O.w")
result = O.dot(witness) == np.multiply(L.dot(witness),R.dot(witness))
assert result.all(), "result contains an inequality"
print(f"verified")

witness= <class 'galois.GF(21888242871839275222246405745257275088548364400416034343698204186575808495617)'>
generated L, R, O R1CS matrices and witness vector
verifying that L.w * R.w == O.w
verified


In [92]:
# R1CS to QAP, interpolate each column of L, R, O
target_poly = GF(np.array([1,2,3,4,5]))

def matrix_to_vector_of_polynomials(matrix):
    return np.apply_along_axis(interpolate_columns, 0, matrix)

def interpolate_columns(column):
    print(column)
    return galois.lagrange_poly(target_poly, column)

# Once interpolation is finished, L, R, O = U.a, V.a, W.a
U_vector_of_poly_x = matrix_to_vector_of_polynomials(L)
V_vector_of_poly_x = matrix_to_vector_of_polynomials(R)
W_vector_of_poly_x = matrix_to_vector_of_polynomials(O)

# L = GF(np.array([
#     [0,0,1,0,0,0,0,0],
#     [0,0,0,0,1,0,0,0],
#     [0,0,0,1,0,0,0,0],
#     [0,0,0,0,0,1,0,0],
#     [0,0,curve_order - 5,0,0,0,0,0] # 113 - 5 = 108
#     ]))

# We interpolate each column. e.g. for L, we interpolate 
# Column 1: [0,0,0,0,0] and target_poly = [1,2,3,4,5]
# Column 2: [0,0,0,0,0] and target_poly = [1,2,3,4,5]
# Column 3: [1,0,0,0,curve_order - 5] and target_poly = [1,2,3,4,5]
# Column 4: [0,0,1,0,0] and target_poly = [1,2,3,4,5]
# Column 5: [0,1,0,0,0] and target_poly = [1,2,3,4,5]
# Column 6: [0,0,0,1,0] and target_poly = [1,2,3,4,5]
# Column 7: [0,0,0,0,0] and target_poly = [1,2,3,4,5]
# Column 8: [0,0,0,0,0] and target_poly = [1,2,3,4,5]

# Resulting in:
#  Column 1: Poly(0)
#  Column 2: Poly(0)
#  Column 3: Poly(2736030358979909402780800718157159386068545550052004292962275523321976061952x^4 + 9120101196599698009269335727190531286895151833506680976540918411073253539842x^3 + 19152212512859365819465605027100115702479818850364030050735928663253832433658x^2 + 12768141675239577212977070018066743801653212566909353367157285775502554955788x + 21888242871839275222246405745257275088548364400416034343698204186575808495612, GF(21888242871839275222246405745257275088548364400416034343698204186575808495617))
#  Column 4: Poly(20064222632519335620392538599819168831169334033714698148390020504361157787649x^4 + 3648040478639879203707734290876212514758060733402672390616367364429301415935x^3 + 12768141675239577212977070018066743801653212566909353367157285775502554955782x^2 + 7296080957279758407415468581752425029516121466805344781232734728858602831863x + 5, GF(21888242871839275222246405745257275088548364400416034343698204186575808495617))
#  Column 5: Poly(0)
#  Column 6: Poly(0)
#  Column 7: Poly(20976232752179305421319472172538221959858849217065366246044112345468483141633x^4 + 9120101196599698009269335727190531286895151833506680976540918411073253539840x^3 + 11856131555579607412050136445347690672963697383558685269503193934395229601794x^2 + 1824020239319939601853867145438106257379030366701336195308183682214650707966x + 1, GF(21888242871839275222246405745257275088548364400416034343698204186575808495617))
#  Column 8: Poly(0)
print(f"the output for each matrix is a vector of polynomials of degree in the number of rows of the matrix")
print(f"a matrix with 5 rows and 8 columns will result in a vector of 8 polynomials of degree 4")

[0 0 0 0 0]
[0 0 0 0 0]
[                                                                            1
                                                                             0
                                                                             0
                                                                             0
 21888242871839275222246405745257275088548364400416034343698204186575808495612]
[0 0 1 0 0]
[0 1 0 0 0]
[0 0 0 1 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[1 1 0 0 0]
[0 0 1 1 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 1]
[0 0 0 0 0]
[                                                                            0
                                                                             0
                                                                             0
                                                                             0
 21888242871839275222246405745257275088548364400416034343698204186575808495615]
[0 0 0 0 1]
[0 0 0 0 0]
[0 0

In [71]:
# Quick Unencrypted Verification
print(f"Quick Unencrypted Verification")
print(f"this is not required but intermediate sanity check")

def reduce_polynomials_to_1(vector_of_poly, witness):
    sum = GF(0)
    for i in range(len(vector_of_poly)):
        sum += vector_of_poly[i]*witness[i]
    return sum

Ua = reduce_polynomials_to_1(U_vector_of_poly_x,witness)
Va = reduce_polynomials_to_1(V_vector_of_poly_x,witness)
Wa = reduce_polynomials_to_1(W_vector_of_poly_x,witness)
print(f"U's vector of polynomials = {U_vector_of_poly_x}")
print(f"")
print(f"U's output from reduction = {Ua}")

print(f"")
target_x = galois.Poly(np.flip(np.polynomial.polynomial.polyfromroots([1,2,3,4,5])).astype(int),  field=GF)
print(f"target_x = {target_x}")
hx, remainder = divmod(((Ua*Va) - Wa), target_x)
print(f"hx = {hx}")
assert remainder == 0, "remainder is not 0"

# Check Tx and Hx balances the polynomial correctly
lhs = Ua * Va
rhs = hx * target_x + Wa
print(f"Verifying U.a * V.a == H(x) * Tx + W.a")
assert lhs == rhs, "result is not balanced across LHS and RHS"
print(f"verified")
print(f"if we evaluate the polynomial at a random tau, wewill have the same result")

Quick Unencrypted Verification
this is not required but intermediate sanity check
U's vector of polynomials = [Poly(0, GF(21888242871839275222246405745257275088548364400416034343698204186575808495617))
 Poly(0, GF(21888242871839275222246405745257275088548364400416034343698204186575808495617))
 Poly(3648040478639879203707734290876212514758060733402672390616367364429301415936x^4 + 10944121435919637611123202872628637544274182200208017171849102093287904247810x^3 + 7296080957279758407415468581752425029516121466805344781232734728858602831868x^2 + 4x, GF(21888242871839275222246405745257275088548364400416034343698204186575808495617))
 Poly(16416182153879456416684804308942956316411273300312025757773653139931856371713x^4 + 21888242871839275222246405745257275088548364400416034343698204186575808495614x^3 + 16416182153879456416684804308942956316411273300312025757773653139931856371725x^2 + 10944121435919637611123202872628637544274182200208017171849102093287904247789x + 10, GF(21888242871839275222246

$[A]_1 = \sum_{i=0}^{m} a_i \cdot u_i(\tau) \cdot G_1$

In [85]:
from functools import reduce
tau = GF(9)
print(f"generate tau = {(tau)}")

# Generate powers of tau
def generate_powers_of_tau(tau, degree, Generator):
    """
    tau is a random num between 1 and curve_order, degree is the highest d of the polynomial + 1 to cater for the constant
    this generates the powers of tau, i.e. tau^0, tau^1, tau^2, tau^3, tau^4, tau^5
    """
    return [multiply(Generator, int(tau ** i)) for i in range(degree + 1)]
 

def inner_product(ec_points, coeffs):
    """
    ec_points is the powers of tao, i.e. [G1, tau*G1, tau^2*G1, tau^3*G1]
    coeffs are the coefficients of the polynomial after lagrange
    this computes the inner product of the 2 vectors i.e. 
    <Ui0, Ui1, Ui2> dot <tau^0, tau^1, tau^2> = x[0] * tau^0 + Ux[1] + tau^1..
    the result is 1 point on the curve representing the whole polynomial
    Z1 is the identity element
    """
    return reduce(add, (multiply(point, int(coeff)) for point, coeff in zip(ec_points, coeffs)), Z1)

powers_of_tau_G1 = generate_powers_of_tau(tau, Ua.degree, G1)
powers_of_tau_G2 = generate_powers_of_tau(tau, Va.degree, G2)
print(f"generated powers of tau. length {len(powers_of_tau_G1)} = {powers_of_tau_G1}")

print(f"generating powers of tau for the target polynomial")
target_poly_x = galois.Poly(np.flip(np.polynomial.polynomial.polyfromroots([1,2,3,4,5])).astype(int),  field=GF)
target_poly_evaluated_tau = target_poly_x(int(tau))
print(f"target polynomial = {(target_poly_x)}")

powers_of_T_tau_G1 = []
for i in range (len(powers_of_tau_G1)):
    powers_of_T_tau_G1.append(multiply(G1, int(target_poly_evaluated_tau)))
print("----",powers_of_T_tau_G1)

h_tau_t_tau = inner_product(powers_of_T_tau_G1, hx.coeffs[::-1])
print(f"generated h_tau_t_tau = {h_tau_t_tau}")


print(f"generating ui(tau), vi(tau) and wi(tau)")

def evaluate_polynomial_at_powers_of_tau(powers_of_tau, vector_of_polynomials):
    poly_evaluated_at_tau = []
    for i in range(len(vector_of_polynomials)):
        # if i == 0:
        #     continue
        iterator_point = inner_product(powers_of_tau, vector_of_polynomials[i].coeffs[::-1])
        poly_evaluated_at_tau.append(iterator_point)
    return poly_evaluated_at_tau
    
U_tau_G1 = evaluate_polynomial_at_powers_of_tau(powers_of_tau_G1, U_vector_of_poly_x)
V_tau_G2 = evaluate_polynomial_at_powers_of_tau(powers_of_tau_G2, V_vector_of_poly_x)
W_tau_G1 = evaluate_polynomial_at_powers_of_tau(powers_of_tau_G1, W_vector_of_poly_x)
print(f"generated ui(tau), vi(tau) and wi(tau)")
print(f"U_tau_G1 = {U_tau_G1}")

witness_integers = np.array([int(element) for element in witness])
print(f"witness as integers: {witness_integers}")

# inner product of U_tau_G1 and witness
print(f"inner product of U, V, W and witness")
U_tau_G1_witness = inner_product(U_tau_G1, witness_integers)
V_tau_G2_witness = inner_product(V_tau_G2, witness_integers)
W_tau_G1_witness = inner_product(W_tau_G1, witness_integers)

print(f"A = U_tau_G1_witness = {U_tau_G1_witness}")
print(f"B = V_tau_G2_witness = {V_tau_G2_witness}")
print(f"C' = W_tau_G1_witness = {W_tau_G1_witness}")
print(f"h(tau)t(tau) = {h_tau_t_tau}")
print(f"C = C' + h(tau)t(tau) = {W_tau_G1_witness + h_tau_t_tau}")
C = add(W_tau_G1_witness, h_tau_t_tau)

print(f"verifying that e(A,B) == e(G2,C)")
eq(pairing(V_tau_G2_witness, U_tau_G1_witness), pairing(G2, C)), "verification failed"




















# print(f"generating powers of tau for target polynomial")
# def generate_powers_of_tau_for_target_poly(tau, T_tau, degree, Generator):
#     return [multiply(Generator, int(T_tau) * int(tau ** i)) for i in range(degree)]

# target_poly_powers_of_tau = generate_powers_of_tau_for_target_poly(tau, target_poly_evaluated_tau, int(target_x.degree), G1)
# print(f"generated powers of tau for target polynomial")

# # target_poly_powers_of_tau_G1 = 
# # scale each ec point by the target poly coefficient
# # for i in range(len(target_poly.coeffs)):

# #     target_poly.coeffs[i] = int(target_poly.coeffs[i])

# # we need to generate h(tau)

# print(f"")
# print("-"*100)
# print(f"Prover Work")
# print(f"Generates h(tau)t(tau)")
# h_tau_t_tau = inner_product(target_poly_powers_of_tau, hx.coeffs[::-1])
# print(f"generated h(tau) = {h_tau_t_tau}")




generate tau = 9
generated powers of tau. length 5 = [(1, 2), (1624070059937464756887933993293429854168590106605707304006200119738501412969, 3269329550605213075043232856820720631601935657990457502777101397807070461336), (11744864753805541320111181058240501346617548818795024075343453303706881344088, 17221384307920533803726063289029126335755140866379809678854881527846933441919), (16614481781419548918333901639087520596453061129401013916352956947099837560149, 6870598320119603643560241397292776091544668466385678757129689863273377343756), (20524137291192142473453794825059764983616394658900507628404688782815983067318, 10777421981167141915367072362265446873927631718869375922147591851415446501221)]
generating powers of tau for the target polynomial
target polynomial = x^5 + 21888242871839275222246405745257275088548364400416034343698204186575808495602x^4 + 85x^3 + 21888242871839275222246405745257275088548364400416034343698204186575808495392x^2 + 274x + 2188824287183927522224640574525727508854836

(False, 'verification failed')

In [72]:
# Compute Generator points and add together
# Uses py_ecc to scale both G1 and G2 points
from functools import reduce
import random

def inner_product(ec_points, coeffs):
    """
    ec_points is the powers of tao, i.e. [G1, tau*G1, tau^2*G1, tau^3*G1]
    coeffs are the coefficients of the polynomial after lagrange
    this computes the inner product of the 2 vectors i.e. 
    <Ui0, Ui1, Ui2> dot <tau^0, tau^1, tau^2> = x[0] * tau^0 + Ux[1] + tau^1..
    the result is 1 point on the curve representing the whole polynomial
    Z1 is the identity element
    """
    print("lenlen len-------",len(ec_points), len(coeffs))
    print(ec_points)
    print(coeffs)
    return reduce(add, (multiply(point, int(coeff)) for point, coeff in zip(ec_points, coeffs)), Z1)


# generate target polynomial (x-1)(x-2)... 
print(f"System Setup")
print(f"generate target polynomial (x-1)(x-2)...")
target_poly_x = galois.Poly(np.flip(np.polynomial.polynomial.polyfromroots([1,2,3,4,5])).astype(int),  field=GF)
print(f"target polynomial = {(target_poly_x)}")

print("-"*100)
# Public Trusted Setup for powers of tau G1, G2
print(f"Trusted Setup")
tau = GF(9)
print(f"generate tau = {(tau)}")
target_poly_tau = target_poly_x(tau)
print(f"target polynomial at tau = {(target_poly_tau)}")

powers_of_tau_G1 = generate_powers_of_tau(tau, Ua.degree, G1)
powers_of_tau_G2 = generate_powers_of_tau(tau, Va.degree, G2)
print(f"powers of tau G1 = {(len(powers_of_tau_G1))}, {(powers_of_tau_G1)}")
print(f"powers of tau G2 = {(len(powers_of_tau_G2))}, {(powers_of_tau_G2)}")

print("-"*100)
# inner product of R1CS matrix and Witness Vector, computed by Prover
print(f"Prover Work")
print(f"Required: [A]1[B]2 = [C] + h(x)t(x)")
print(f"Prover has Ua, Va, Wa from previous step (evaluation of R1CS Matrix + Witness in single polynomial form)")
print(f"prover generates [A]1 [B]2 [C]1 by inner product with tau")
inner_product_ua_and_tau = inner_product(powers_of_tau_G1, Ua.coeffs[::-1])
inner_product_va_and_tau = inner_product(powers_of_tau_G2, Va.coeffs[::-1])
inner_product_wa_and_tau = inner_product(powers_of_tau_G1, Wa.coeffs[::-1])

print(f"")
# [A][B] = [C] + h(x)t(x)
A1 = inner_product_ua_and_tau
B2 = inner_product_va_and_tau

print(f"computing  h(x)t(x)")
hx, remainder = divmod((Ua * Va - Wa), target_poly_x)
assert remainder == 0, "remainder is not 0"
print(f"h(x)) = ((Ua * Va - Wa) // t(x)) = {(hx)}")
ht = hx * target_poly_x
print(f"h(x)t(x) = {(ht)}")
inner_product_ht_and_tau = inner_product(powers_of_tau_G1, ht.coeffs[::-1])
print(f"prover generates [C1 + h(x)t(x)] by inner product with tau")
C1 = add(inner_product_wa_and_tau, inner_product_ht_and_tau)

print(f"test pairing([A1]1[B2]2) == pairing([C1]1[G2]2) ")
if(eq(pairing(B2,A1), pairing(G2,C1))):
    print("true")
else:
    print("false")


System Setup
generate target polynomial (x-1)(x-2)...
target polynomial = x^5 + 21888242871839275222246405745257275088548364400416034343698204186575808495602x^4 + 85x^3 + 21888242871839275222246405745257275088548364400416034343698204186575808495392x^2 + 274x + 21888242871839275222246405745257275088548364400416034343698204186575808495497
----------------------------------------------------------------------------------------------------
Trusted Setup
generate tau = 9
target polynomial at tau = 6720
powers of tau G1 = 5, [(1, 2), (1624070059937464756887933993293429854168590106605707304006200119738501412969, 3269329550605213075043232856820720631601935657990457502777101397807070461336), (11744864753805541320111181058240501346617548818795024075343453303706881344088, 17221384307920533803726063289029126335755140866379809678854881527846933441919), (16614481781419548918333901639087520596453061129401013916352956947099837560149, 68705983201196036435602413972927760915446684663856787571296898632733

Problem: prover can provide any [A][B][C] points to pass proof verification
Solution: require prover add α & β shift to [A][B]. 
Trusted setup computes 

Trusted Setup provides 
1. [α]G1
2. [β]G2
3. powers of tau for G1 and G2

Trusted Setup computes
1. U(τ)
2. V(τ)
3. W(τ)
4. T(τ)
5. H(τ)

Then computes
1. [βU(τ) + αV(τ) + W(τ)]G1
2. [H(τ)T(τ)]G1

In [75]:
# Trusted Setup
alpha = 2
beta = 3
alpha_G1 = multiply(G1, alpha)
beta_G2 = multiply(G2, beta)

print(f"Goal: [A+α]G1[B+β]G2")
print(f"[α]G1 = {(alpha_G1)}")
print(f"[β]G2 = {(beta_G2)}")
A1_new = add(alpha_G1, A1)
B2_new = add(beta_G2, B2)
print(f"[A+α]G1 = {(A1_new)}")
print(f"[B+β]G2 = {(B2_new)}")
print(f"")

print(f"Goal: C = ai[βUi(τ^i) + αVi(τ^i) + Wi(τ^i)]G1 + [H(τ)T(τ)]G1")
print(f"")
print(f"First, let's work on lhs: C = ai[βUi(τ) + αVi(τ) + Wi(τ)]G1 ")

print(f"Trusted Setup creates: [βUi(τ) + αVi(τ) + Wi(τ)]G1 =  [βU0(τ) + αV0(τ) + W0(τ)]G1, [βU1(τ) + αV1(τ) + W1(τ)]G1 ...")

powers_of_tau_for_C = []
for i in range (len(L_vector_of_poly)):
    U_i = L_vector_of_poly[i]
    V_i = R_vector_of_poly[i]
    W_i = O_vector_of_poly[i]
    result = beta * U_i(tau**i) + alpha * V_i(tau**i) + W_i(tau**i)
    temp_c = multiply(G1, int(result))
    powers_of_tau_for_C.append(temp_c)
print(f" C_powers_of_tau = {(powers_of_tau_for_C)}")

print(f"Prover then computes: ai[C_powers_of_tau[i]]G1 = a0[C_powers_of_tau[0]]G1 + a1[C_powers_of_tau[1]]G1 ...")
inner_product_powers_of_tau_for_c_G1 = inner_product(powers_of_tau_for_C, wgf)
print(f"ai[βUi(τ) + αVi(τ) + Wi(τ)]G1 = {(inner_product_powers_of_tau_for_c_G1)}")

print(f"")
print(f"next work on rhs: [H(τ)T(τ)]G1 = h0[τ^0 * t(τ) * G1] + h1[τ^1 * t(τ) * G1]... where h is the coeff of h(x). Result is 1 G1 Point")
print(f"prover has h(x) from before: {(hx)}")
target_poly_evaluated_at_tau = target_poly_x(tau)
target_poly_G1 = [multiply(G1,int(tau**i)) for i in range(0,target_poly_x.degree-1)]
print(f"target poly G1 = {(target_poly_G1)}")
HT = inner_product(target_poly_G1, hx.coeffs[::-1])
print(f"HT inner product of t(x) coeffs and target poly at G1= {(HT)}")
print(f"")

print(f"Finally, prover computes: C = ai[βUi(τ) + αVi(τ) + Wi(τ)]G1 + [H(τ)T(τ)]G1")
C1_new = add(inner_product_powers_of_tau_for_c_G1, HT)
print(f"C1 = {(C1_new)}")

if(pairing(B2_new, A1_new) == pairing(G2, C1_new) * pairing(beta_G2, alpha_G1)):
    print("true")
else:
    print("false")

Goal: [A+α]G1[B+β]G2
[α]G1 = (1368015179489954701390400359078579693043519447331113978918064868415326638035, 9918110051302171585080402603319702774565515993150576347155970296011118125764)
[β]G2 = ((2725019753478801796453339367788033689375851816420509565303521482350756874229, 7273165102799931111715871471550377909735733521218303035754523677688038059653), (2512659008974376214222774206987427162027254181373325676825515531566330959255, 957874124722006818841961785324909313781880061366718538693995380805373202866))
[A+α]G1 = (20096163336571570922849981278144060791661189853715098484148184730778481231840, 6627583918305965613148819179838670156417744549394555615527314261843149574175)
[B+β]G2 = ((706014206693897160366191714856177545161153092615017185243497322461220958346, 16228311717649579475864504096988153203773770404102010454517633001967547588814), (19763269111005225995105725183003882283451659105765012301033858061111094482959, 1583662909152268266787495617565350434821461836389573582044220766836306609